In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('data_by_location/Kasurila_data.csv') # Error 8.88
#df = pd.read_csv('data_by_location/Levi_data.csv') # Error 17.05
#df = pd.read_csv('data_by_location/Luosto_data.csv') # Error 13.97
#df = pd.read_csv('data_by_location/Messila_data.csv') # Error 6.61
#df = pd.read_csv('data_by_location/Mustavaara_data.csv') # Error 9.38
#df = pd.read_csv('data_by_location/Ounasvaara_data.csv') # Error 12.27
#df = pd.read_csv('data_by_location/Purnu_data.csv') # Error 8.10
#df = pd.read_csv('data_by_location/Ruka_data.csv') # Error 11.46
#df = pd.read_csv('data_by_location/Ruunarinteet_data.csv') # Error 7.28
#df = pd.read_csv('data_by_location/Salla_data.csv') # Error 13.26

In [12]:
df.columns

Index(['date', 'snow_depth_cm', 'avg_temp_c', 'uv_index', 'cloud_cover_rate',
       'cloud_cover', 'location'],
      dtype='object')

In [13]:
df.sample(15)

,date,snow_depth_cm,avg_temp_c,uv_index,cloud_cover_rate,cloud_cover,location
374,2004-03-18,NaN,0.7,NaN,NaN,NaN,Kasurila
5658,2018-09-05,0.0,15.1,0.4,5.0,Partly cloudy,Kasurila
1249,2006-08-10,0.0,18.6,1.1,3.0,Mostly clear,Kasurila
7426,2023-07-09,0.0,14.5,0.9,8.0,Cloudy,Kasurila
2553,2010-03-06,76.0,-9.7,0.1,8.0,Cloudy,Kasurila
4250,2014-10-28,0.0,9.3,0.0,7.0,Mostly cloudy,Kasurila
489,2004-07-11,NaN,16.7,NaN,NaN,NaN,Kasurila
5514,2018-04-14,65.0,5.7,0.6,0.0,Clear,Kasurila
4348,2015-02-03,46.0,-2.1,0.0,8.0,Cloudy,Kasurila
3864,2013-10-07,0.0,10.0,0.1,8.0,Cloudy,Kasurila


In [14]:
df.cloud_cover.value_counts()

cloud_cover
Clear                              2432
Mostly cloudy                      2142
Cloudy                             1888
Partly cloudy                       362
Mostly clear                        283
Cloudiness cannot be determined      78
Name: count, dtype: int64

In [15]:
df.replace(to_replace='-', value=np.nan, inplace=True)

In [16]:
df.snow_depth_cm = df.snow_depth_cm.replace(to_replace='-1', value='0')

In [17]:
df.cloud_cover.value_counts()

cloud_cover
Clear                              2432
Mostly cloudy                      2142
Cloudy                             1888
Partly cloudy                       362
Mostly clear                        283
Cloudiness cannot be determined      78
Name: count, dtype: int64

In [19]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [20]:
df.columns

Index(['date', 'snow_depth_cm', 'avg_temp_c', 'uv_index', 'cloud_cover_rate',
       'cloud_cover', 'location'],
      dtype='object')

In [21]:
df.dtypes

date                 object
snow_depth_cm       float64
avg_temp_c          float64
uv_index            float64
cloud_cover_rate    float64
cloud_cover          object
location             object
dtype: object

In [ ]:
df.avg_temp_c.value_counts()

avg_temp_c
1.7      73
0.5      72
0.7      69
0.3      68
0.9      68
         ..
-28.4     1
-24.7     1
-28       1
-22.1     1
-20.5     1
Name: count, Length: 557, dtype: int64

In [ ]:
df.snow_depth_cm.value_counts()

snow_depth_cm
0     5839
2      124
3      120
5      108
4      108
      ... 
85       1
84       1
95       1
93       1
91       1
Name: count, Length: 90, dtype: int64

In [22]:
# Remove duplicates
df = df.drop_duplicates(subset=['date'], keep='last')

In [23]:
df['avg_temp_c_float'] = pd.to_numeric(df['avg_temp_c'], errors='coerce')

In [24]:
df['uv_index_float'] = pd.to_numeric(df['uv_index'], errors='coerce')

In [25]:
df = df.dropna(subset=['avg_temp_c_float'])
df = df.dropna(subset=['snow_depth_cm'])

In [26]:
# Only values after 2006
selector = df['date'] >= '2006-01-01'
df = df[df['date'] >= '2006-01-01']

In [27]:
# Create lag features

df['snow_depth_1d_ago'] = df.groupby('location')['snow_depth_cm'].shift(1)

df['snow_depth_7d_ago'] = df.groupby('location')['snow_depth_cm'].shift(7)

df['snow_depth_365d_ago'] = df.groupby('location')['snow_depth_cm'].shift(365)
 

In [28]:
# Convert date string to datetime object
df['date'] = pd.to_datetime(df['date'])
    
# Extract time-based features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_year'] = df['date'].dt.dayofyear

In [29]:
# Create an XGBoost regressor model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

X = df[['avg_temp_c_float', 'cloud_cover_rate','uv_index_float', 'snow_depth_1d_ago', 'snow_depth_7d_ago', 'snow_depth_365d_ago', 'year', 'month', 'day', 'day_of_year']]  # Features 
y = df['snow_depth_cm']  # Target variable (snow depth)

# Split into training (80%) and test (20%) data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [30]:
predictions = model.predict(X_test)

In [32]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error: {mae}")

mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Absolute Error: 0.8017129374371449
Mean Squared Error: 2.8338800852226096
